In [1]:
import bs4
from datetime import datetime
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import pandas as pd
from selenium import webdriver                    # Import module 
from selenium.webdriver.common.keys import Keys   # For keyboard keys 
import time                                       # Waiting function  
import matplotlib.pyplot as plt

news_url="https://news.google.com/news/rss/search?q=crude+oil&num=400"
Client=urlopen(news_url)
xml_page=Client.read()
Client.close()

soup_page=soup(xml_page,"xml")
news_list=soup_page.findAll("item")
print(len(news_list))
# Print news title, url and publish date

titles = []
dates = []

for news in news_list:
  titles.append(news.title.text)
  dates.append(datetime.strptime(news.pubDate.text, '%a, %d %b %Y %H:%M:%S %Z'))

100


In [ ]:
browser = webdriver.Safari() 
for page in range (0,10):
    browser.get('https://www.businesstimes.com.sg/search/crude%20oil?page=' + str(page))       # 1 
    time.sleep(5) 
    # 2 
    data = browser.find_elements_by_class_name("media-body") 

    for i in range (0, len(data)):
        titles.append((data[i].find_element_by_tag_name("a")).text)
        dates.append(datetime.strptime(((data[i].find_element_by_tag_name("time")).text),'%d %b %Y'))

df = pd.DataFrame({'Date':dates,'Titles':titles})

print(df.tail)


In [ ]:
# Import Libraries

from textblob import TextBlob
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
nltk.download('vader_lexicon')
import pycountry
import re
import string

from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#Sentiment Analysis

def percentage(part,whole):
    return 100 * float(part)/float(whole) 

positive  = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []

In [ ]:
#Cleaning Text (RT, Punctuation etc)
#Creating new dataframe and new features
tw_list = df
tw_list["text"] = tw_list["Titles"]

In [ ]:
#Calculating Negative, Positive, Neutral and Compound values
tw_list[['polarity', 'subjectivity']] = tw_list['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tw_list['text'].iteritems():
 score = SentimentIntensityAnalyzer().polarity_scores(row)
 neg = score['neg']
 neu = score['neu']
 pos = score['pos']
 comp = score['compound']
 if neg > pos:
    tw_list.loc[index, 'sentiment'] = "negative"
 elif pos > neg:
    tw_list.loc[index, 'sentiment'] = "positive"
 else:
    tw_list.loc[index, 'sentiment'] = "neutral"
    tw_list.loc[index, 'neg'] = neg
    tw_list.loc[index, 'neu'] = neu
    tw_list.loc[index, 'pos'] = pos
    tw_list.loc[index, 'compound'] = comp
    
tw_list.head(10)

In [ ]:
#Creating new data frames for all sentiments (positive, negative and neutral)
tw_list_negative = tw_list[tw_list["sentiment"]=="negative"]
tw_list_positive = tw_list[tw_list["sentiment"]=="positive"]
tw_list_neutral = tw_list[tw_list["sentiment"]=="neutral"]

In [ ]:
def count_values_in_column(data,feature):
 total=data.loc[:,feature].value_counts(dropna=False)
 percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
 return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])
#Count_values for sentiment
count_values_in_column(tw_list,"sentiment")

In [ ]:
# create data for Pie Chart
pichart = count_values_in_column(tw_list,"sentiment")
names= pichart.index
size=pichart["Percentage"]

#Creating PieCart
labels = ['Positive ['+str(size[1])+'%]' , 'Neutral ['+str(0)+'%]','Negative ['+str(size[0])+'%]']
sizes = [12,0, 178]
colors = ['green', 'lightblue','red']
patches, texts = plt.pie(sizes,colors=colors, startangle=90)
plt.style.use('default')
plt.legend(labels)
plt.title("Sentiment Analysis of Crude Oil news articles")
plt.axis('equal')
plt.show()

In [ ]:
df.plot(x='Date', y='polarity')

#df.drop(['Titles', 'text', 'subjectivity', 'subjectivity' , 'sent' ], axis=1)

In [ ]:
df.drop(['Titles','text','subjectivity','sentiment'] , axis=1, inplace=True)
df.tail()

In [ ]:
df.drop(['neg','neu','pos','compound'], axis=1, inplace=True)
df.tail()


In [ ]:
df.set_index('Date', inplace=True, drop=True)
df.tail()

In [ ]:
df.index = pd.DatetimeIndex(df.index)


In [ ]:
df.tail()

In [ ]:
df = df[~df.index.duplicated(keep='first')]

In [ ]:
idx = pd.date_range(min(df.index),max(df.index))

df = df.reindex(idx, fill_value=0)

print(df)

import matplotlib.pyplot as plt
plt.plot(df.index,df["polarity"], label = "you cuk too")

plt.legend()
plt.show()

In [ ]:
# create data for Pie Chart
pichart = count_values_in_column(tw_list,"sentiment")
names= pichart.index
size=pichart["Percentage"]

#Creating PieCart
labels = ['Positive ['+str(size[1])+'%]' , 'Neutral ['+str(0)+'%]','Negative ['+str(size[0])+'%]']
sizes = [positive, negative]
colors = ['green', 'lightblue','red']
patches, texts = plt.pie(sizes,colors=colors, startangle=90)
plt.style.use('default')
plt.legend(labels)
plt.title("Sentiment Analysis Result for keyword= "+keyword+"" )
plt.axis('equal')
plt.show()